## 1\. Концепция и философия: Зачем мы вообще это делаем?

Представь, что мы строим не просто базу данных, а цифровую модель реального мира. В реальном мире у нас есть не только числа и строки, но и такие понятия, как "адрес", "денежная сумма", "статус заказа" или "координаты GPS".

Пользовательские типы позволяют нам перенести эти концепции прямо в схему базы данных. Вместо того чтобы говорить: "Это поле `TEXT`, но мы договорились хранить в нем email", мы говорим: "Это поле имеет тип `email_address`".

**Какие ключевые проблемы это решает?**

1.  **Семантика и читаемость:** Схема становится самодокументируемой. `delivery_address address` говорит гораздо больше, чем `delivery_address_line1 TEXT, delivery_address_city TEXT, ...`. Это повышает понятность и снижает порог входа для новых разработчиков.

2.  **Абстракция:** Мы скрываем сложность. Вместо того чтобы работать с набором из пяти столбцов для адреса, мы работаем с одной сущностью — `address`. Это упрощает запросы и логику на стороне приложения.

3.  **Целостность и верификация данных:** Мы переносим логику валидации с уровня приложения на уровень базы данных — туда, где ей и место. Если у нас есть тип `positive_integer`, база данных сама не позволит записать в него отрицательное число, независимо от того, кто и как пытается это сделать. Это фундаментальный принцип надежности.

### Сравнение подходов: UDT vs. Отдельные таблицы vs. JSONB

Это классический архитектурный выбор. Давай разберем на примере хранения адреса (`street`, `city`, `postal_code`).

| Критерий | Пользовательский тип (COMPOSITE) | Отдельная таблица (`addresses`) | `JSONB` |
| :--- | :--- | :--- | :--- |
| **Сценарий** | Адрес — это неотъемлемый *атрибут* сущности (например, адрес доставки заказа). Он не существует сам по себе. | Адреса — это *отдельные сущности*, которые могут переиспользоваться (например, один адрес у нескольких пользователей). | Нужна максимальная гибкость схемы; структура адреса может сильно меняться. |
| **Связь** | 1 к 1 (Один заказ — один адрес доставки). | 1 ко многим / многие ко многим (Один пользователь — много адресов). | 1 к 1. |
| **Производительность** | 🚀 **Очень высокая.** Данные хранятся вместе с основной строкой. Нет `JOIN`. | 🐢 **Медленнее.** Требуется `JOIN` для получения данных, что увеличивает нагрузку на I/O. | 🐇 **Быстро, но с нюансами.** Нет `JOIN`, но есть накладные расходы на парсинг JSON. |
| **Целостность данных** | ✅ **Высокая.** Структура типа жестко определена. | ✅ **Высокая.** Структура таблицы и связи обеспечивают целостность. | ❌ **Низкая.** Нет гарантии, что все JSON-объекты будут иметь одинаковые ключи и типы значений. |
| **Индексация** | ✅ Да, можно индексировать поля внутри типа. | ✅ Да, можно индексировать любой столбец. | ✅ Да, с помощью GIN-индексов, но они специфичны. |
| **Вердикт** | **Идеально** для инкапсуляции связанных данных, которые являются атрибутом, а не самостоятельной сущностью. | **Стандартный выбор** для нормализованных данных и связей "один ко многим". | **Хорошо** для "полуструктурированных" данных, где схема может меняться, но за это приходится платить целостностью. |

**Золотое правило:** если данные не имеют смысла в отрыве от родительской сущности — это кандидат на `COMPOSITE TYPE`. Если они могут существовать самостоятельно и ссылаться из разных мест — это кандидат на отдельную таблицу.

-----

## 2\. Типы типов: Инструментарий архитектора

А теперь давай посмотрим, какие инструменты есть в нашем арсенале.

### Составные типы (COMPOSITE)

Это аналог `struct` в C/Go или `class` без методов в Python/Java. Он группирует несколько полей в единое целое.

**Пример:** Давай для нашей условной базы данных `northwind` создадим тип для хранения денежных сумм, который будет включать значение и валюту.

```sql
-- 1. Создаем сам тип
CREATE TYPE money_amount AS (
    amount   NUMERIC(10, 2),
    currency CHAR(3)
);

-- 2. Используем его в таблице
-- Представим, что мы рефакторим таблицу products
ALTER TABLE products ADD COLUMN price money_amount;

-- Обновим данные для одного продукта
UPDATE products
SET price = ROW(25.50, 'USD') -- Используем конструктор ROW() для создания значения
WHERE product_id = 1;

-- 3. Как с этим работать?
-- Обращение к полям через точку
SELECT 
    product_name, 
    (price).amount, 
    (price).currency
FROM products 
WHERE (price).amount > 20;

-- Можно получить и весь объект
SELECT product_name, price FROM products WHERE product_id = 1;
-- Результат будет выглядеть как: (25.50,USD)
```

**Зачем это нужно?** Мы гарантируем, что цена *всегда* идет вместе с валютой. Невозможно указать одно без другого. Это семантически правильно и безопасно.

### Перечисления (ENUM)

ENUM — это тип данных, который может принимать только одно из заранее определенных строковых значений.

**Плюсы:**

  * **Производительность и хранение:** На диске ENUM хранится как целое число (обычно 4 байта), а не как строка. Это экономит место и ускоряет сравнения и `JOIN`.
  * **Читаемость и безопасность:** `order_status = 'shipped'` гораздо понятнее, чем `order_status_id = 3`. И вы не сможете присвоить ему некорректное значение.

**Минусы:**

  * **Ригидность:** Изменение набора значений — это целая история. Об этом ниже.
  * **Сортировка:** Сортировка происходит в порядке объявления значений в `CREATE TYPE`, что не всегда интуитивно (например, если добавить новое значение в середину).

**Пример:** Статус заказа — идеальный кандидат для ENUM.

```sql
-- 1. Создаем тип
CREATE TYPE order_status AS ENUM (
    'pending',      -- Ожидает обработки
    'processing',   -- В обработке
    'shipped',      -- Отправлен
    'delivered',    -- Доставлен
    'cancelled'     -- Отменен
);

-- 2. Используем в таблице orders
ALTER TABLE orders ADD COLUMN status order_status DEFAULT 'pending';

-- 3. Работаем с ним
UPDATE orders SET status = 'shipped' WHERE order_id = 10248;

-- Это не сработает и вызовет ошибку, что прекрасно!
-- UPDATE orders SET status = 'lost_in_mail' WHERE order_id = 10249;
```

**Когда использовать ENUM?** Когда у вас есть небольшой, стабильный и редко изменяемый набор состояний. Статусы, типы объектов, категории с фиксированным списком.

**Когда НЕ использовать ENUM?** Для тегов, категорий товаров в интернет-магазине или любых других списков, которые могут часто пополняться пользователями. В этом случае лучше использовать классическую справочную таблицу.

### Псевдонимы типов (DOMAIN)

DOMAIN — это не новый тип данных, а существующий тип с "обвесом" в виде ограничений. Это твой главный инструмент для централизации бизнес-правил.

**Чем отличается от `CHECK CONSTRAINT` на колонке?**
`CHECK CONSTRAINT` живет и умирает вместе со столбцом. Если у тебя 10 таблиц, где есть поле "email", тебе придется в 10 местах написать `CHECK (email ~* '...@...[.]...')`. А если завтра регулярное выражение для проверки email изменится? Тебе придется менять его в 10 местах.

`DOMAIN` решает эту проблему. Ты создаешь правило один раз.

**Пример:** Создадим домен для почтового индекса США.

```sql
-- 1. Создаем домен на основе TEXT
CREATE DOMAIN us_postal_code AS TEXT
    -- Ограничение на уровне домена
    CHECK (
        VALUE ~ '^\d{5}$' OR VALUE ~ '^\d{5}-\d{4}$'
    )
    NOT NULL;

-- 2. Используем его в таблицах
ALTER TABLE customers ADD COLUMN postal_code us_postal_code;
ALTER TABLE suppliers ADD COLUMN postal_code us_postal_code;
ALTER TABLE employees ADD COLUMN postal_code us_postal_code;

-- 3. Попытка вставить некорректные данные
-- Эта команда вызовет ошибку, и это защищает наши данные на самом низком уровне
-- INSERT INTO customers (customer_id, company_name, postal_code) 
-- VALUES ('TEST', 'Test Inc.', '1234'); 
```

Теперь, если нам понадобится изменить формат почтового индекса, мы изменим его в одном месте — в определении `DOMAIN`. Вся база данных мгновенно начнет работать по новому правилу. 

---

## 3\. JSON и JSONB

В PostgreSQL есть два типа для хранения JSON-данных: `JSON` и `JSONB`. На первый взгляд они кажутся одинаковыми, но под капотом это совершенно разные звери.

**Золотое правило:** **Почти всегда используйте `JSONB`**. Выбирайте `JSON` только если у вас есть веская причина хранить JSON-текст в его *исходном, нетронутом виде*.

| Характеристика | `JSON` (текстовый) | `JSONB` (бинарный) |
| :--- | :--- | :--- |
| **Хранение** | Хранит точную текстовую копию. Пробелы, порядок ключей, дубликаты ключей — всё сохраняется. | Хранит данные в разобранном бинарном формате. Пробелы удаляются, порядок ключей не гарантируется, из дубликатов сохраняется только последний. |
| **Скорость записи (INSERT)** | 🚀 **Очень быстрая.** Нет накладных расходов, просто сохранение текста. | 🐢 **Медленнее.** Требуется время на парсинг и преобразование в бинарный формат. |
| **Скорость чтения (SELECT)** | 🐢 **Очень медленная.** Текст нужно парсить при каждом обращении к внутренним элементам. | 🚀 **Очень быстрая.** Данные уже разобраны, доступ к элементам почти мгновенный. |
| **Индексация** | ❌ **Нет.** Невозможно создать индекс для поиска по внутренним элементам JSON-документа. | ✅ **Да\!** Поддерживает мощные GIN-индексы, что делает поиск внутри документов невероятно эффективным. |
| **Основной сценарий** | Хранение логов или данных из внешних API, где важна побайтовая точность оригинала. | Хранение и активная работа с JSON-документами: фильтрация, извлечение данных, модификация. |

### Практика: Операторы и функции `JSONB`

Представим, что у нас есть таблица товаров с `JSONB`-колонкой для хранения характеристик.

```sql
CREATE TABLE products (
    id SERIAL PRIMARY KEY,
    name TEXT,
    attributes JSONB
);

INSERT INTO products (name, attributes) VALUES
('Ноутбук ProX', '{"brand": "TechCorp", "ram_gb": 16, "is_touchscreen": false, "specs": {"cpu": "i7", "cores": 8}}'),
('Смартфон Z10', '{"brand": "ConnectMe", "ram_gb": 8, "is_touchscreen": true, "tags": ["popular", "bestseller"] }');
```

**1. Извлечение данных**

  * `->` возвращает поле как `JSONB`.
  * `->>` возвращает поле как `TEXT`. Это ключевое различие\!

<!-- end list -->

```sql
-- Получить бренд как текст
SELECT name, attributes ->> 'brand' AS brand FROM products;

-- Получить вложенный объект specs как JSONB
SELECT name, attributes -> 'specs' AS specs FROM products;
```

**2. Глубокое извлечение (по пути)**

  * `#>` возвращает поле по пути как `JSONB`.
  * `#>>` возвращает поле по пути как `TEXT`.

<!-- end list -->

```sql
-- Получить модель CPU как текст
SELECT name, attributes #>> '{specs,cpu}' AS cpu_model
FROM products
WHERE name = 'Ноутбук ProX';
```

**3. Проверка и фильтрация (здесь `JSONB` раскрывается полностью)**

  * `@>` (содержит): Проверяет, содержится ли левый операнд в правом.
  * `?` (существует): Проверяет, существует ли ключ верхнего уровня.
  * `?|` (существует один из): Проверяет, существует ли хотя бы один из ключей в массиве.
  * `?&` (существуют все): Проверяет, существуют ли все ключи в массиве.

<!-- end list -->

```sql
-- Найти все товары бренда "TechCorp"
SELECT name FROM products WHERE attributes @> '{"brand": "TechCorp"}';

-- Найти все товары с сенсорным экраном
SELECT name FROM products WHERE attributes @> '{"is_touchscreen": true}';

-- Найти все товары, у которых есть тег "popular"
SELECT name FROM products WHERE attributes -> 'tags' @> '["popular"]';

-- Найти все товары, у которых есть ключ "ram_gb"
SELECT name FROM products WHERE attributes ? 'ram_gb';
```

### Индексация `JSONB`

Без индекса все эти запросы будут приводить к полному сканированию таблицы (Full Table Scan). Чтобы сделать их быстрыми, используется GIN-индекс.

```sql
-- Создаем GIN-индекс на всю колонку attributes
CREATE INDEX idx_products_attributes_gin ON products USING GIN (attributes);
```

Теперь запросы с операторами `@>`, `?`, `?|`, `?&` будут работать практически мгновенно даже на миллионах строк. Этот индекс — главная причина, по которой `JSONB` так популярен.

---

## 4\. `ALTER TYPE` и безопасное изменение типов

Изменение типа данных в работающей базе — одна из самых рискованных операций. Она может заблокировать таблицы на долгое время или привести к потере данных. Вот как делать это правильно для разных видов типов.

### 4.1. Изменение `ENUM`

Это самая коварная часть, особенно с ENUM.

* **Добавление значения в ENUM:** Это относительно безопасная операция, но с нюансами.

  ```sql
  -- Добавляем новое значение 'returned' в конец списка
  ALTER TYPE order_status ADD VALUE 'returned';

  -- Можно добавить до или после существующего значения
  -- ALTER TYPE order_status ADD VALUE 'on_hold' BEFORE 'shipped';
  ```

  **Подводный камень:** Эта команда требует эксклюзивной блокировки таблицы на короткое время. В высоконагруженных системах это нужно делать в период низкой активности.
  
  **Профессиональный совет:** В высоконагруженных системах используйте `SET lock_timeout`. Если команда не может получить блокировку за указанное время, она отменится, а не будет "вешать" всю систему.

* **Безопасные миграции для ENUM в продакшене.** Это критически важно. Чтобы добавить новое значение в ENUM без простоя (zero-downtime):

  * **Шаг 1:** В отдельной, неблокирующей транзакции выполни `ALTER TYPE ... ADD VALUE ...`. Эта команда быстрая, но требует короткой блокировки.
    ```sql
    -- Запускать в отдельном сеансе
    SET lock_timeout = '2s'; -- Не ждать блокировки вечно
    ALTER TYPE order_status ADD VALUE 'awaiting_pickup' AFTER 'shipped';
    ```
  * **Шаг 2:** Выкати новую версию приложения, которая *знает* об этом новом значении и умеет его обрабатывать. Старые версии приложения продолжат работать, они просто не будут "видеть" это значение.
  * **Шаг 3:** После того как все инстансы приложения обновлены, можно начинать использовать новое значение.

* **Переименование значения в ENUM:**

  ```sql
  ALTER TYPE order_status RENAME VALUE 'pending' TO 'awaiting_payment';
  ```

* **Удаление значения из ENUM:** **Это невозможно\!** PostgreSQL не позволяет удалять значения из ENUM, потому что непонятно, что делать с уже существующими данными в таблицах, которые используют это значение. Единственный путь — создать новый тип, перенести данные, удалить старый столбец со старым типом и переименовать новый. Это сложная и опасная миграция.

  1.  **Создаем новый временный ENUM:**
      ```sql
      CREATE TYPE order_status_new AS ENUM ('pending', 'processing', 'shipped', 'cancelled'); -- Без 'delivered'
      ```
  2.  **Добавляем новый столбец в таблицу:**
      ```sql
      ALTER TABLE orders ADD COLUMN status_new order_status_new;
      ```
  3.  **Копируем и преобразуем данные:**
      ```sql
      UPDATE orders
      SET status_new = status::text::order_status_new
      WHERE status != 'delivered';
      -- Решаем, что делать с удаляемым значением 'delivered'. Например, меняем на 'shipped'.
      UPDATE orders
      SET status_new = 'shipped'
      WHERE status = 'delivered';
      ```
  4.  **В одной транзакции меняем столбцы местами:**
      ```sql
      BEGIN;
      ALTER TABLE orders DROP COLUMN status;
      ALTER TABLE orders RENAME COLUMN status_new TO status;
      COMMIT;
      ```
  5.  **Удаляем старый тип:**
      ```sql
      DROP TYPE order_status;
      ```

  Это сложная миграция, требующая окна обслуживания и тщательного тестирования.

### 4.2. Изменение `DOMAIN`

Здесь всё гораздо проще. Домены созданы для централизованного управления правилами.

  * **Добавление/удаление `NOT NULL`:**

    ```sql
    ALTER DOMAIN us_postal_code SET NOT NULL;
    ALTER DOMAIN us_postal_code DROP NOT NULL;
    ```

    При добавлении `SET NOT NULL` PostgreSQL проверит все существующие данные. Если найдет `NULL`, операция завершится ошибкой.

  * **Управление ограничениями `CHECK`:**

    ```sql
    -- Добавляем новое, более строгое ограничение
    ALTER DOMAIN us_postal_code ADD CONSTRAINT valid_format_strict CHECK (VALUE ~ '^\d{5}-\d{4}$');

    -- Удаляем старое ограничение
    ALTER DOMAIN us_postal_code DROP CONSTRAINT us_postal_code_check;
    ```

    При добавлении нового `CONSTRAINT` база данных также проверит все столбцы, использующие этот домен. Это мощнейший инструмент для повышения качества данных во всей системе одной командой.

### 4.3. Изменение `COMPOSITE TYPE`

Составные типы довольно гибки.

  * **Добавление/Удаление/Переименование атрибута:**

    ```sql
    -- Добавляем поле
    ALTER TYPE money_amount ADD ATTRIBUTE country_code CHAR(2);

    -- Удаляем поле
    ALTER TYPE money_amount DROP ATTRIBUTE country_code;

    -- Переименовываем поле
    ALTER TYPE money_amount RENAME ATTRIBUTE amount TO sum;
    ```

    Эти операции обычно быстрые, так как являются изменениями метаданных.

  * **Изменение типа данных атрибута:**

    ```sql
    ALTER TYPE money_amount ALTER ATTRIBUTE sum TYPE NUMERIC(12, 4);
    ```

    **Внимание\!** Эта операция ведет себя так же, как `ALTER TABLE ... ALTER COLUMN`. Если изменение типа требует перестройки данных на диске (например, `TEXT` в `INTEGER`), это приведет к **полной перезаписи всех таблиц**, использующих этот тип. На больших таблицах это означает длительную блокировку `ACCESS EXCLUSIVE` и простой в работе.

### 4.4. "Изменение" схемы `JSONB`

`JSONB` не имеет жесткой схемы на уровне СУБД, поэтому `ALTER TYPE` к нему неприменим. "Миграция" схемы `JSONB` — это просто `UPDATE`-запросы.

  * **Добавление нового ключа всем документам:**

    ```sql
    UPDATE products
    SET attributes = attributes || '{"in_stock": true}'::jsonb
    WHERE attributes ->> 'in_stock' IS NULL;
    ```

  * **Переименование ключа:**

    ```sql
    UPDATE products
    SET attributes = (attributes - 'brand') || jsonb_build_object('manufacturer', attributes -> 'brand')
    WHERE attributes ? 'brand';
    ```

    Здесь мы удаляем старый ключ `brand` и добавляем новый `manufacturer` со значением из старого.

  * **Изменение типа данных значения внутри JSONB:**
    Предположим, `ram_gb` хранился как строка `"16"`, а мы хотим сделать его числом `16`.

    ```sql
    UPDATE products
    SET attributes = jsonb_set(
        attributes,
        '{ram_gb}',
        to_jsonb((attributes ->> 'ram_gb')::INT)
    )
    WHERE jsonb_typeof(attributes -> 'ram_gb') = 'string';
    ```

    Здесь мы используем `jsonb_set` для обновления значения по пути `{ram_gb}`, предварительно преобразовав текстовое значение в `INTEGER`, а затем обратно в `JSONB`-число.

**Ключевой вывод:** Гибкость `JSONB` переносит ответственность за целостность схемы с базы данных на вас. Миграции данных внутри `JSONB` требуют написания аккуратных `UPDATE`-скриптов и не дают тех гарантий, которые предоставляют строгие типы.

---

## 5\. Подводные камни и Best Practices 

### Производительность: как это хранится?

  * **Составные типы (COMPOSITE):** PostgreSQL очень эффективен. Он хранит значение составного типа *inline*, то есть прямо внутри строки (tuple) на диске.  Когда ты делаешь `SELECT * FROM orders`, все поля адреса доставки считываются за одну дисковую операцию вместе с остальными данными заказа. Это кардинально быстрее, чем делать `JOIN` с отдельной таблицей `addresses`, что потребовало бы как минимум еще одной дисковой операции для поиска и считывания строки из другой таблицы.

  * **Сравнение с JSONB:** `JSONB` тоже хранится inline, но в бинарном, распарсенном формате. При чтении поля из `JSONB` все равно есть небольшие накладные расходы на "распаковку" нужного ключа, в то время как у `COMPOSITE` типа поля имеют фиксированное смещение, что делает доступ к ним практически мгновенным.

### Индексация: ключ к скорости

Да, поля внутри составных типов можно и нужно индексировать\! Это одна из самых крутых фич.

**Пример:** Допустим, мы часто ищем клиентов по почтовому индексу, который находится внутри составного типа `address`.

```sql
-- Сначала создадим тип и таблицу
CREATE TYPE full_address AS (
    city        TEXT,
    street      TEXT,
    postal_code TEXT
);

CREATE TABLE clients (
    client_id SERIAL PRIMARY KEY,
    name      TEXT,
    shipping_address full_address
);

-- Создаем индекс по полю postal_code внутри типа shipping_address
CREATE INDEX idx_clients_postal_code 
ON clients (((shipping_address).postal_code)); -- Обрати внимание на синтаксис!

-- Теперь этот запрос будет использовать индекс и выполняться молниеносно
EXPLAIN ANALYZE
SELECT name, shipping_address 
FROM clients 
WHERE (shipping_address).postal_code = '10115';
```

### Взаимодействие с клиентским кодом

Современные драйверы умеют работать с пользовательскими типами, преобразуя их в нативные структуры языка.

  * **Python (`psycopg`):** Этот драйвер может автоматически "регистрировать" составные типы и маппить их на `collections.namedtuple` или на твои кастомные классы. Это делает код чистым и типобезопасным. Вместо `row['address'][2]` ты пишешь `row.address.postal_code`.

  * **Go (`pgx`):** `pgx` имеет отличную поддержку UDT. Он может сканировать результаты запроса напрямую в ваши структуры (`struct`). Это устраняет необходимость ручного парсинга и делает код очень идиоматичным.

### Метаданные: где Postgres хранит информацию о типах?

Вся информация о типах хранится в системном каталоге, в основном в таблице `pg_type`.

```sql
-- Посмотреть все пользовательские типы (не системные)
SELECT 
    n.nspname AS schema_name,
    t.typname AS type_name,
    t.typtype AS type_category -- b=base, c=composite, d=domain, e=enum
FROM pg_type t
JOIN pg_namespace n ON n.oid = t.typnamespace
WHERE n.nspname NOT IN ('pg_catalog', 'information_schema');
```

  * Если `typtype = 'c'` (composite), то можно найти соответствующую "таблицу" в `pg_class`, чтобы посмотреть атрибуты.
  * Если `typtype = 'e'` (enum), то можно посмотреть все его возможные значения в таблице `pg_enum`.

### **Совместимость с ORM.** 
Это больное место. Большинство крупных ORM (SQLAlchemy, Hibernate, Django ORM) имеют базовую поддержку UDT, но часто она требует написания дополнительного кода-прослойки для маппинга.

* **Domains** обычно поддерживаются лучше всего, так как для ORM это просто базовый тип с дополнительной проверкой.
* **ENUMs** тоже, как правило, хорошо поддерживаются.
* **Composite Types** — самая сложная область. Перед тем как активно их использовать, обязательно проверь документацию своей ORM и напиши небольшой proof-of-concept, чтобы убедиться, что все работает как надо (чтение, запись, миграции схемы).